In [7]:
import tensorflow as tf
from tensorflow import keras

import keras_tuner as kt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten
import warnings
from google.colab import drive

ImportError: Traceback (most recent call last):
  File "c:\Users\rajua\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
img_height, img_width = 224, 224
batch_size = 32
folder_path = 'D:\Mini_project_sem6\TONGUE IMAGE\中医舌诊染苔数据'

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    horizontal_flip=True,
    zoom_range=0.2
)

train_generator = datagen.flow_from_directory(
    folder_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    folder_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 1615 images belonging to 2 classes.
Found 403 images belonging to 2 classes.


In [ ]:
# Function to build the model with tunable hyperparameters
def build_model(hp):
    from tensorflow.keras.layers import Dropout # Import Dropout here
    model = Sequential()

    # First Conv Layer
    model.add(Conv2D(
        filters=hp.Choice('conv1_filters', [32, 64]),  # Tune number of filters
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(img_height, img_width, 3)
    ))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Second Conv Layer
    model.add(Conv2D(
        filters=hp.Choice('conv2_filters', [64, 128]),
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Third Conv Layer
    model.add(Conv2D(
        filters=hp.Choice('conv3_filters', [128, 256]),
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Dropout Layer to Reduce Overfitting
    model.add(Dropout(hp.Float('dropout_rate', 0.2, 0.5, step=0.1)))  # Tune dropout rate

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(
        units=hp.Choice('dense_units', [64, 128, 256]),  # Tune dense layer size
        activation='relu'
    ))

    model.add(Dense(2, activation='softmax'))  # Output layer

    # Compile Model with Tunable Learning Rate
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
# Initialize Keras Tuner (Random Search)
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuner_results',
    project_name='cnn_tuning'
)


Reloading Tuner from tuner_results/cnn_tuning/tuner0.json


In [ ]:
# Start Hyperparameter Search
tuner.search(train_generator, validation_data=validation_generator, epochs=5)  # Replace with your data

# Get Best Model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best Dropout Rate: {best_hps.get('dropout_rate')}")
print(f"Best Learning Rate: {best_hps.get('learning_rate')}")
print(f"Best Dense Layer Size: {best_hps.get('dense_units')}")

In [ ]:
# Train the Best Model
best_model = tuner.hypermodel.build(best_hps)


In [ ]:

history = best_model.fit(train_data, validation_data=val_data, epochs=20)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from tensorflow.keras.utils import to_categorical

epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()

In [ ]:


# Get true labels and predictions
y_true = np.argmax(y_test, axis=1)  # Convert one-hot to class labels
y_pred_probs = model.predict(x_test)  # Get probability predictions
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert to class labels

# Compute Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

# Plot Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:

# Compute ROC Curve
fpr, tpr, _ = roc_curve(y_test[:, 1], y_pred_probs[:, 1])  # Use probability of class 1
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Random chance line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()
